In [3]:
import math
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

drone_width = 12

class node:
    # (x,y) is the center of the building, z = height of the buildng
    # w, h are x_length and y_length of the building
    def __init__(self, tag, x, y, z, w, h):
        self.tag = tag
        self.x = x
        self.y = y
        self.z = z
        self.w = w
        self.h = h
        self.approachable = []
        
    def add_node(self, n):
        self.approachable.append(n)
        

def checkRange(node1, node2, node3):
    rect = []
    x_cord = [node3.x - 0.6*node3.w, node3.x + 0.6*node3.w]
    y_cord = [node3.y - 0.6*node3.h, node3.y + 0.6*node3.h]
    
    for x in x_cord:
        for y in y_cord:
            rect.append((x,y))

    if node1.x <= node2.x:
        n1 = node1
        n2 = node2
    else:
        n1 = node2
        n2 = node1
        
    p1 = (n1.x - drone_width, n1.y - drone_width)
    p2 = (n1.x + drone_width, n1.y + drone_width)
    p3 = (n2.x - drone_width, n2.y - drone_width)
    p4 = (n2.x + drone_width, n2.y + drone_width)
    
    poly1 = Polygon([p1, p2, p3, p4])
    
    poly2 = Polygon(rect)
    
    if poly1.intersects(poly2):
        return True
    
    return False
        

def getDistance(node1, node2):
    return math.sqrt( ((node1.x-node2.x)**2) + ((node1.y-node2.y)**2) )


In [5]:
import csv

file = open('BuildingPosition_noblanks.csv', 'r')
data = csv.reader(file)

node_list = []

for line in data:
    new_node = node(line[0], float(line[1]), float(line[2]), float(line[3]), float(line[4]), float(line[5]))
    node_list.append(new_node)

file.close()

ValueError: invalid literal for int() with base 10: '1.58'

In [184]:
for n in node_list:
    for n_1 in node_list:
        
        # Ignore if they are the same node
        if n == n_1:
            continue
            
        distance = getDistance(n, n_1)  
        is_blocked = False
        
        for n_2 in node_list:
            if n_2 == n_1 or n_2 == n:
                continue
             
            is_blocked = checkRange(n, n_1, n_2)  
            
            if is_blocked:   
                # If lower/equalto both buildings, it is not blocking
                if n_2.z < n.z and n_2.z <= n_1.z:
                    is_blocked = False
                    
                # If higher than both buildings, it is blocked
                elif n_2.z > n.z and n_2.z > n_1.z:
                    break;          
                    
                else:
                    if n.z <= n_1.z:
                        low = n
                        high = n_1
                    else:
                        low = n_1
                        high = n
                                 
                    # The mid-building's height is in between two other buildings
                    poly = Polygon([(low.x, low.z), (high.x, low.z), (high.x, high.z)])
                    
                    if poly.contains(Point(n_2.x, n_2.z)):
                        is_blocked = False
                    else: 
                        break;

        if not is_blocked:
            n.add_node((n_1, round(distance,2)))
    
    n.approachable.sort(key = lambda x: x[0].tag)

In [185]:
node_list.sort(key = lambda x: x.tag)
writable_lists = []
for source in node_list:
    for dest in source.approachable:
        writable_list = []
        writable_list.append(('Source', source.tag))
        writable_list.append(('Dest', dest[0].tag))
        writable_list.append(('Dist', dest[1]))
        writable_list.append(('X', source.x))
        writable_list.append(('Y', source.y))
        writable_list.append(('Z', source.z))
        writable_lists.append(writable_list)
    

In [186]:
with open('node_map.csv', 'w', newline='') as file:
    fieldnames = ['Source', 'Dest', 'Dist', 'X', 'Y', 'Z']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for obj in writable_lists:
        toDict = dict(obj)
        writer.writerow(toDict)